In [1]:
!nvidia-smi

Mon Sep 28 23:39:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   40C    P0    37W / 300W |    198MiB / 16125MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   41C    P0    38W / 300W |      0MiB / 16128MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
!pip install keras2onnx pycuda

     |################################| 96 kB 4.8 MB/s eta 0:00:011
     |################################| 81 kB 15.5 MB/s eta 0:00:01
     |################################| 64 kB 6.4 MB/s  eta 0:00:01
     |################################| 7.4 MB 8.7 MB/s eta 0:00:01
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=11f8b16663771436409d380f2bb05e9a835a3e5d60b371036882be0a21aabcdf
  Stored in directory: /root/.cache/pip/wheels/3e/38/8f/e5bd4465e4b72c8e4d1530ac46e1a42e1d2495f8c93242d5f1
Successfully built fire
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
from tensorflow.keras.applications import ResNet50

In [4]:
model = ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 5s 0us/step


In [5]:
import numpy as np

dummy_input_batch = np.zeros((32, 224, 224, 3))

model.predict(dummy_input_batch) # warm up

array([[1.69642444e-04, 3.30074545e-04, 6.13502853e-05, ...,
        1.46223765e-05, 1.44498641e-04, 6.60868129e-04],
       [1.69642444e-04, 3.30074545e-04, 6.13502853e-05, ...,
        1.46223765e-05, 1.44498641e-04, 6.60868129e-04],
       [1.69642444e-04, 3.30074545e-04, 6.13502853e-05, ...,
        1.46223765e-05, 1.44498641e-04, 6.60868129e-04],
       ...,
       [1.69642444e-04, 3.30074545e-04, 6.13502853e-05, ...,
        1.46223765e-05, 1.44498641e-04, 6.60868129e-04],
       [1.69642532e-04, 3.30074574e-04, 6.13503144e-05, ...,
        1.46223847e-05, 1.44498656e-04, 6.60868187e-04],
       [1.69642444e-04, 3.30074545e-04, 6.13503144e-05, ...,
        1.46223920e-05, 1.44498728e-04, 6.60868129e-04]], dtype=float32)

In [6]:
%%time

print("Warming up...")

model.predict(dummy_input_batch) # Check default performance

print("Done warming up!")

Warming up...
Done warming up!
CPU times: user 11.4 s, sys: 1.05 s, total: 12.4 s
Wall time: 432 ms


In [7]:
import onnx, keras2onnx

In [8]:
onnx_model = keras2onnx.convert_keras(model, model.name)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 458 -> 127


In [9]:
model_name = "resnet50_onnx_model.onnx"
onnx.save_model(onnx_model, model_name)

In [ ]:
import os

os._exit(0) # Shut down all kernels so TRT doesn't fight with Tensorflow for GPU memory - TF monopolizes all GPU memory by default

In [1]:
# May need to shut down all kernels and restart before this:
!trtexec --onnx=resnet50_onnx_model.onnx --batch=1 --saveEngine=resnet_engine.trt  --explicitBatch 2>&1

&&&& RUNNING TensorRT.trtexec # trtexec --onnx=resnet50_onnx_model.onnx --batch=1 --saveEngine=resnet_engine.trt --explicitBatch
[09/28/2020-23:40:28] [I] === Model Options ===
[09/28/2020-23:40:28] [I] Format: ONNX
[09/28/2020-23:40:28] [I] Model: resnet50_onnx_model.onnx
[09/28/2020-23:40:28] [I] Output:
[09/28/2020-23:40:28] [I] === Build Options ===
[09/28/2020-23:40:28] [I] Max batch: explicit
[09/28/2020-23:40:28] [I] Workspace: 16 MB
[09/28/2020-23:40:28] [I] minTiming: 1
[09/28/2020-23:40:28] [I] avgTiming: 8
[09/28/2020-23:40:28] [I] Precision: FP32
[09/28/2020-23:40:28] [I] Calibration: 
[09/28/2020-23:40:28] [I] Safe mode: Disabled
[09/28/2020-23:40:28] [I] Save engine: resnet_engine.trt
[09/28/2020-23:40:28] [I] Load engine: 
[09/28/2020-23:40:28] [I] Builder Cache: Enabled
[09/28/2020-23:40:28] [I] NVTX verbosity: 0
[09/28/2020-23:40:28] [I] Inputs format: fp32:CHW
[09/28/2020-23:40:28] [I] Outputs format: fp32:CHW
[09/28/2020-23:40:28] [I] Input build shapes: model
[09/28

In [5]:
import numpy as np

dummy_input_batch = np.zeros((32, 224, 224, 3))

In [6]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

f = open("resnet_engine.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

In [7]:
output = np.empty(1000, dtype = np.float32)

# Allocate device memory
d_input = cuda.mem_alloc(1 * dummy_input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

In [8]:
def predict_batch(batch): # result gets copied into output
    # Transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # Execute model
    context.execute_async(1, bindings, stream.handle, None)
    # Transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # Syncronize threads
    stream.synchronize()

In [9]:
print("Warming up...")

predict_batch(dummy_input_batch)

print("Done warming up!")

Warming up...
Done warming up!


In [10]:
%%time

predict_batch(dummy_input_batch) # Check TRT performance

print ("Prediction: " + str(np.argmax(output)))

Prediction: 111
CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 8.15 ms
